In [ ]:
import tensorflow as tf
import keras_tuner as kt
import generator as my_generator
import preprocess as my_preprocess
import model as my_modeler
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [ ]:
class_number = 2
clip_length_seconds = 4
number_of_samples = 800 
test_ratio = 0.2
validation_ratio = 0.2 
batch_size = 4
languages = ['japanese', 'english']
languages_dir = ['audio/japanese',
                  'audio/english']
datamaker  = my_preprocess.DatasetMaker(number_of_samples)
train_df = datamaker.make_sample_dataframe(languages, languages_dir)
train_df, test_df = datamaker.split_dataframe(train_df, test_ratio)
train_df, validation_df = datamaker.split_dataframe(train_df, validation_ratio)

#### Hypermodel Tuning

In [ ]:
epochs = 10
hp = kt.HyperParameters()
hypermodel = my_modeler.MyHyperModel(train_df, validation_df, class_number, clip_length_seconds)

tuner = kt.RandomSearch(hypermodel=hypermodel,
                        objective='val_binary_accuracy',
                        max_trials=20,
                        executions_per_trial=1,
                        overwrite=False,
                        directory='searchResults',
                        project_name='audioClassification2')

tuner.search(epochs=epochs)                                                          

In [ ]:
from contextlib import redirect_stdout
best_hp = tuner.get_best_hyperparameters()[0]
with open('searchResults/audioClassification/results.txt', 'w') as f:
  with redirect_stdout(f):
    tuner.results_summary(num_trials=40)

#### Ordinary Model Training

In [ ]:
train_generator = my_generator.MyGenerator(train_df, 
                                            batch_size, 
                                            clip_length_seconds, 
                                            spec_nfft=2**8,
                                            spec_window=2**8,
                                            spec_stride=2**7,
                                            spec_mels=240)
validation_generator = my_generator.MyGenerator(validation_df,
                                                batch_size, 
                                                clip_length_seconds, 
                                                spec_nfft=2**8,
                                                spec_window=2**8,
                                                spec_stride=2**7,
                                                spec_mels=240)
image_size = validation_generator.spectrogram_shape

# Model parameters
learning_rate = 0.0001
batch_size = 4
epochs = 25

# Model
modeler = my_modeler.MyModeler(image_size, class_number) 
model = modeler.make_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.BinaryAccuracy()]
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y_%m_%d-%H:%M:%S') 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_generator, validation_data=validation_generator, epochs=epochs, callbacks = [tensorboard_callback])


In [ ]:
model.save("models/model.h5")
model.save("models/model")